# Star Jeans _ Product pricing project based on competition

## Business problem
- What is the best selling price for pants? 

## Questions to be answered:
- What is the best selling price for the pants?
- How many types of pants and their colors for the initial product?
- What are the raw materials needed to make the pants?

## Data source
- H&M: https://www2.hm.com/en_us/men/products/jeans.html
- Macys: https://www.macys.com/shop/mens-clothing/mens-jeans

# Imports

In [24]:
# !pip install requests
# !pip install bs4

In [2]:
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import numpy as np
import re
import sqlite3 
from sqlalchemy import create_engine

# Loading data

In [36]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
           
page = requests.get( url, headers=headers )

In [37]:
soup = BeautifulSoup( page.text, 'html.parser' )

In [38]:
products = soup.find( 'ul', class_='products-listing small' )

In [39]:
product_list = products.find_all( 'article', class_='hm-product-item')

# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]

# product category
product_category = [p.get( 'data-category' ) for p in product_list]

# product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

# price
product_list = products.find_all( 'span', class_='price regular' )
product_price = [p.get_text() for p in product_list]

In [40]:
data = pd.DataFrame( [product_id, product_category, product_name,product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name','product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )

In [41]:
total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get('data-total')
total_item

'85'

In [42]:
page_number = np.ceil( int( total_item ) / 36 )
page_number

3.0

In [43]:
url02 = url + '?page-size=' + str( int( page_number*36 ) )
url02

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=108'

In [49]:

# empty dataframe
df_details = pd.DataFrame()

# unique columns for all products
aux = []
cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']
df_pattern = pd.DataFrame( columns=cols )

for i in range( len( data ) ):
    
# API Requests
        url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
        page = requests.get( url, headers=headers )

        # Beautiful Soup object
        soup = BeautifulSoup( page.text, 'html.parser' )

        # ==================== color name =================================
        product_list = soup.find_all( 'a', class_='filter-option miniature' )
        color_name = [p.get( 'data-color' ) for p in product_list]

        # product id
        product_id = [p.get( 'data-articlecode' ) for p in product_list]
        df_color = pd.DataFrame( [product_id, color_name] ).T
        df_color.columns = ['product_id', 'color_name']

        # generate style id + color id
        df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3] )
        df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:] )

# ==================== composition =================================

        product_composition_list = soup.find_all( 'div',class_='pdp-description-list-item' )
        product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

        # reaname dataframe
        df_composition = pd.DataFrame( product_composition ).T
        df_composition.columns = df_composition.iloc[0]

        # delete first row
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )

        # garantee the same number of columns
        df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

        # generate style id + color id
        df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3] )
        df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:] )
        aux = aux + df_composition.columns.tolist()

        # merge data color + decomposition
        data_sku = pd.merge( df_color, df_composition[['style_id', 'Fit','Composition', 'Size', 'Product safety']], how='left', on='style_id' )

        # all details products
        df_details = pd.concat( [df_details, data_sku], axis=0 )

                                                                                                       
# Join Showroom data + details
data['style_id'] = data['product_id'].apply( lambda x: x[:-3] )
data['color_id'] = data['product_id'].apply( lambda x: x[-3:] )
data_raw = pd.merge( data, df_details[['style_id', 'color_name', 'Fit','Composition', 'Size', 'Product safety']], how = 'left', on='style_id')

data_raw.to_csv('Datasets/data_raw_star_jeans.csv', index=False)

IndexError: single positional indexer is out-of-bounds

# Clear Data

In [50]:
data = pd.read_csv('Datasets/data_raw_star_jeans.csv')

# product id
data = data.dropna( subset=['product_id'] )
data['product_id'] = data['product_id'].astype( int )

# product name
data['product_name'] = data['product_name'].apply( lambda x: x.replace( ' ','_' ).lower() )

# product price
data['product_price'] = data['product_price'].apply( lambda x: x.replace( '$ ','' ) ).astype( float )

# scrapy datetime
data['scrapy_datetime'] = pd.to_datetime( data['scrapy_datetime'],format='%Y-%m-%d %H:%M:%S' )

# style id
data['style_id'] = data['style_id'].astype( int )

# color id
data['color_id'] = data['color_id'].astype( int )

# color name
data['color_name'] = data['color_name'].apply( lambda x: x.replace( ' ', '_' ).replace( '/', '_' ).lower() 
                                              if pd.notnull( x ) else x )

# fit
data['Fit'] = data['Fit'].apply( lambda x: x.replace( ' ', '_' ).lower() if pd.notnull( x ) else x )

# size number
data['size_number'] = data['Size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0)
                                         if pd.notnull( x ) else x )

data['size_number'] = data['Size'].apply( lambda x: re.search( '\d+', x ).group(0) if pd.notnull( x ) else x )

# size model
data['size_model'] = data['Size'].str.extract( '(\d+/\\d+)' )

# composition

### drop Pocket lining,Lining, Shell

data=data[~data['Composition'].str.contains('Pocket lining:',na=False)]
data=data[~data['Composition'].str.contains('Lining:',na=False)]
data=data[~data['Composition'].str.contains('Shell:',na=False)]
data=data[~data['Composition'].str.contains('Pocket:',na=False)]

# Drop duplicates
data=data.drop_duplicates(subset=['product_id', 'product_category', 'product_name', 'product_price',
      'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'Fit'],keep='last')

#Reset index
data=data.reset_index(drop=True)


# break composition by comma
df1=data['Composition'].str.split(',', expand=True)


# ##############################Creating a collumn for each matterials#####################################

# cotton/polyester/Elastano/Elasterell

df_ref= pd.DataFrame(index=np.arange(len(data)),columns=['cotton','polyester','elastane','elasterell'])

# Cotton
df_cotton=df1[0]
df_cotton.name = 'cotton'

df_ref= pd.concat([df_ref,df_cotton],axis=1)
df_ref=df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')] # elimina e mantem a ultima coluna 
df_ref['cotton']=df_ref['cotton'].fillna('Cotton 0%')
    
# Polyester

df_polyester= df1.loc[df1[1].str.contains('Polyester',na=True),1]
df_polyester.name = 'polyester'
df_ref= pd.concat([df_ref,df_polyester],axis=1)
df_ref=df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')] # elimina e mantem a ultima coluna 
df_ref['polyester']=df_ref['polyester'].fillna('Polyester 0%')

# # Elastano
df_elastane= df1.loc[df1[1].str.contains('Elastane',na=True),1]
df_elastane.name = 'elastane'

df_elastane=df_elastane.combine_first(df1[2])
df_ref= pd.concat([df_ref,df_elastane],axis=1)
df_ref=df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')] # elimina e mantem a ultima coluna 
df_ref['elastane']=df_ref['elastane'].fillna('Elastane 0%')


# # Elasterell
df_elasterell= df1.loc[df1[1].str.contains('Elasterell',na=True),1]
df_elasterell.name = 'elasterell'

df_ref= pd.concat([df_ref,df_elasterell],axis=1)
df_ref=df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')] # elimina e mantem a ultima coluna 
df_ref['elasterell']=df_ref['elasterell'].fillna('Elasterell 0%')


# # #Final join
data=pd.concat([data, df_ref], axis=1)
data['cotton']=data['cotton'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)
data['elastane']=data['elastane'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)
data['polyester']=data['polyester'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)
data['elasterell']=data['elasterell'].apply(lambda x: int(re.search('\d+',x).group(0))/100 if pd.notnull(x) else x)

#Drop columns
data = data.drop( columns=['Size', 'Product safety', 'Composition'], axis=1 )
data.drop_duplicates()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,light_denim_blue_trashed,skinny_fit,187,32/32,0.98,0.00,0.02,0.00
1,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,denim_blue,skinny_fit,187,32/32,0.98,0.00,0.02,0.00
2,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,light_denim_blue,skinny_fit,187,32/32,0.98,0.00,0.02,0.00
3,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,black_washed_out,skinny_fit,187,32/32,0.98,0.00,0.02,0.00
4,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,white,skinny_fit,187,32/32,0.98,0.00,0.02,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,1004476002,men_jeans_slim,freefit®_slim_jeans,49.99,2021-09-23 15:38:15,1004476,2,dark_denim_blue,slim_fit,NaN,NaN,0.90,0.00,0.02,0.08
111,1004476002,men_jeans_slim,freefit®_slim_jeans,49.99,2021-09-23 15:38:15,1004476,2,black_no_fade_black,slim_fit,NaN,NaN,0.90,0.00,0.02,0.08
112,1013317004,men_jeans_regular,hybrid_regular_tapered_joggers,39.99,2021-09-23 15:38:15,1013317,4,black,regular_fit,189,NaN,0.78,0.21,0.01,0.00
113,1013317004,men_jeans_regular,hybrid_regular_tapered_joggers,39.99,2021-09-23 15:38:15,1013317,4,light_denim_blue,regular_fit,NaN,NaN,0.78,0.21,0.01,0.00


In [51]:
data.to_csv('Datasets/products_hm_cleaned.csv', index=False)

# Data Base Sqlite

In [8]:
data = pd.read_csv('Datasets/products_hm_cleaned.csv')

In [9]:
query_showroom_schema = '''
    CREATE TABLE showroom (
        product_id        INTEGER, 
        product_category  TEXT, 
        product_name      TEXT,
        product_price     REAL,
        scrapy_datetime   TEXT, 
        style_id          INTEGER, 
        color_id          TEXT, 
        color_name        TEXT, 
        Fit               TEXT,
        size_number       REAL, 
        size_model        TEXT, 
        cotton            REAL, 
        polyester         REAL, 
        elastane          REAL,
        elasterell        REAL
  
    )
    
'''

In [10]:
#connect to data base 

conn = sqlite3.connect ('hm_db.sqlite')
conn.execute(query_showroom_schema)
conn.commit()
conn.close()

OperationalError: table showroom already exists

In [11]:
conn=create_engine('sqlite:///hm_db.sqlite',echo=False)

In [15]:
query = """
    SELECT* FROM showroom
"""
df = pd.read_sql_query( query, conn )

In [16]:
# insert data to table
data.to_sql( 'showroom', con=conn, if_exists='append', index=False )

In [17]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,light_denim_blue_trashed,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
1,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,denim_blue,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
2,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,light_denim_blue,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
3,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,black_washed_out,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
4,690449022,men_jeans_ripped,skinny_jeans,39.99,2021-09-23 15:38:15,690449,22,white,skinny_fit,187.0,32/32,0.98,0.0,0.02,0.0
